In [7]:
# ========================================
# AI PODCAST INTELLIGENCE SYSTEM
# JupyterLab Production Version
# ========================================

# CELL 1: Import Dependencies and Setup
import os
import sys
import logging
import yaml
import json
import sqlite3
import feedparser
import requests
import pandas as pd
import webbrowser
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, List, Optional, Tuple
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [8]:
# CELL 2: Load Configuration and Verify Setup
print("\n🔧 LOADING CONFIGURATION...")

try:
    # Load YAML configuration
    with open('podcast_config.yaml', 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    
    # Get API key from environment
    anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
    
    if anthropic_api_key:
        print("✅ Configuration loaded successfully")
        print("✅ Anthropic API key found")
    else:
        print("❌ Anthropic API key not found in .env file")
        print("Please add your API key to the .env file")
    
    # Show active podcasts
    active_podcasts = {name: cfg for name, cfg in config['podcasts'].items() if cfg.get('active', True)}
    print(f"📻 Active podcasts configured: {len(active_podcasts)}")
    
    for name, cfg in active_podcasts.items():
        print(f"   • {name} ({cfg.get('priority', 'medium')} priority)")
    
    # Estimate weekly cost
    total_episodes = sum(cfg.get('estimated_episodes_per_week', 1) for cfg in active_podcasts.values())
    estimated_cost = total_episodes * 0.03  # Rough estimate
    print(f"💰 Estimated weekly cost: ~${estimated_cost:.2f}")
    
except Exception as e:
    print(f"❌ Configuration error: {e}")
    config = None
    anthropic_api_key = None


🔧 LOADING CONFIGURATION...
✅ Configuration loaded successfully
✅ Anthropic API key found
📻 Active podcasts configured: 8
   • Practical AI (high priority)
   • Last Week in AI (high priority)
   • The AI Podcast (medium priority)
   • Lex Fridman Podcast (high priority)
   • Eye on AI (high priority)
   • AI Today Podcast (medium priority)
   • The AI Show (medium priority)
   • MIT AI Podcast (medium priority)
💰 Estimated weekly cost: ~$0.27


In [28]:
# CELL 3: Production AI Podcast Intelligence System Class - Complete Enhanced Version
class ProductionPodcastIntelligence:
    """Production-ready AI Podcast Intelligence System for JupyterLab"""
    
    def __init__(self, config_dict: Dict):
        self.config = config_dict
        self.db_path = config_dict['system']['database_path']
        self.reports_dir = Path(config_dict['system']['reports_directory'])
        self.logs_dir = Path(config_dict['system']['logs_directory'])
        self.anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
        
        # Create directories if they don't exist
        self.reports_dir.mkdir(exist_ok=True)
        self.logs_dir.mkdir(exist_ok=True)
        Path(self.db_path).parent.mkdir(exist_ok=True)
        
        # Setup logging
        self.setup_logging()
        
        # Initialize database
        self.setup_database()
        
        # Update database schema for enhanced features
        self.update_database_schema()
        
        self.logger.info(f"Production system initialized with {len(self.get_active_podcasts())} active podcasts")
    
    def setup_logging(self):
        """Setup production logging"""
        log_file = self.logs_dir / f"podcast_intelligence_{datetime.now().strftime('%Y%m%d')}.log"
        
        # Configure logging
        logging.basicConfig(
            level=getattr(logging, self.config['system']['log_level']),
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(log_file),
                logging.StreamHandler(sys.stdout)
            ]
        )
        
        self.logger = logging.getLogger('PodcastIntelligence')
    
    def setup_database(self):
        """Create database tables with enhanced schema"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Episodes table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS episodes (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                podcast_name TEXT NOT NULL,
                title TEXT NOT NULL,
                pub_date TEXT,
                description TEXT,
                audio_url TEXT,
                episode_url TEXT,
                duration_minutes INTEGER,
                guid TEXT,
                processed BOOLEAN DEFAULT 0,
                processing_attempts INTEGER DEFAULT 0,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                UNIQUE(podcast_name, title)
            )
        ''')
        
        # Intelligence table with enhanced fields
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS intelligence (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                episode_id INTEGER,
                headline_takeaway TEXT,
                executive_summary TEXT,
                strategic_implications TEXT,
                technical_developments TEXT,
                market_dynamics TEXT,
                key_people TEXT,
                companies_mentioned TEXT,
                predictions TEXT,
                actionable_insights TEXT,
                risk_factors TEXT,
                quantified_impact TEXT,
                bottom_line TEXT,
                guest_expertise TEXT,
                importance_score INTEGER,
                confidence_score REAL,
                processing_cost REAL,
                processing_time_seconds REAL,
                model_used TEXT,
                episode_url TEXT,
                processed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (episode_id) REFERENCES episodes (id)
            )
        ''')
        
        # Processing logs table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS processing_logs (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                operation_type TEXT,
                podcast_name TEXT,
                episode_title TEXT,
                status TEXT,
                error_message TEXT,
                cost REAL,
                processing_time REAL,
                timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Reports table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS reports (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                report_type TEXT,
                report_period TEXT,
                episodes_count INTEGER,
                total_cost REAL,
                file_path TEXT,
                generated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        conn.commit()
        conn.close()
        self.logger.info("Database setup completed")
    
    def update_database_schema(self):
        """Update database schema to include all enhanced fields"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        try:
            # Check existing columns in episodes table
            cursor.execute("PRAGMA table_info(episodes)")
            existing_columns = {column[1] for column in cursor.fetchall()}
            
            # Add missing columns to episodes table
            new_episode_columns = {
                'episode_url': 'TEXT',
                'guid': 'TEXT',
                'duration_minutes': 'INTEGER'
            }
            
            for column, data_type in new_episode_columns.items():
                if column not in existing_columns:
                    cursor.execute(f'ALTER TABLE episodes ADD COLUMN {column} {data_type}')
                    self.logger.info(f"Added {column} column to episodes table")
            
            # Check existing columns in intelligence table
            cursor.execute("PRAGMA table_info(intelligence)")
            existing_intel_columns = {column[1] for column in cursor.fetchall()}
            
            # Add missing columns to intelligence table
            new_intel_columns = {
                'headline_takeaway': 'TEXT',
                'strategic_implications': 'TEXT',
                'market_dynamics': 'TEXT',
                'actionable_insights': 'TEXT',
                'risk_factors': 'TEXT',
                'quantified_impact': 'TEXT',
                'bottom_line': 'TEXT',
                'guest_expertise': 'TEXT',
                'episode_url': 'TEXT'
            }
            
            for column, data_type in new_intel_columns.items():
                if column not in existing_intel_columns:
                    cursor.execute(f'ALTER TABLE intelligence ADD COLUMN {column} {data_type}')
                    self.logger.info(f"Added {column} column to intelligence table")
            
            conn.commit()
            conn.close()
            
        except Exception as e:
            self.logger.error(f"Error updating database schema: {e}")
            conn.close()
    
    def get_active_podcasts(self) -> Dict:
        """Get active podcasts from configuration"""
        return {name: cfg for name, cfg in self.config['podcasts'].items() if cfg.get('active', True)}
    
    def fetch_recent_episodes(self, podcast_name: str, podcast_config: Dict) -> List[Dict]:
        """Enhanced episode fetching with better URL extraction"""
        try:
            self.logger.info(f"Fetching episodes from {podcast_name}")
            feed = feedparser.parse(podcast_config['rss_url'])
            
            if not feed.entries:
                self.logger.warning(f"No episodes found for {podcast_name}")
                return []
            
            days_back = self.config['system']['days_lookback']
            max_episodes = self.config['system']['max_episodes_per_podcast']
            since_date = datetime.now() - timedelta(days=days_back)
            
            new_episodes = []
            for entry in feed.entries[:max_episodes]:
                try:
                    # Parse publication date
                    pub_date = datetime(*entry.published_parsed[:6]) if hasattr(entry, 'published_parsed') else datetime.now()
                    
                    if pub_date >= since_date:
                        # Enhanced URL extraction
                        episode_url = self.extract_episode_url(entry)
                        audio_url = self.extract_audio_url(entry)
                        
                        episode_data = {
                            'podcast_name': podcast_name,
                            'title': entry.title,
                            'pub_date': pub_date.isoformat(),
                            'description': getattr(entry, 'description', '') or getattr(entry, 'summary', ''),
                            'audio_url': audio_url,
                            'episode_url': episode_url,
                            'duration_minutes': self.extract_duration(entry),
                            'guid': getattr(entry, 'guid', getattr(entry, 'id', ''))
                        }
                        new_episodes.append(episode_data)
                        
                        self.logger.debug(f"Episode URL found: {episode_url}")
                        
                except Exception as e:
                    self.logger.error(f"Error processing episode from {podcast_name}: {e}")
                    continue
            
            self.logger.info(f"Found {len(new_episodes)} new episodes from {podcast_name}")
            return new_episodes
            
        except Exception as e:
            self.logger.error(f"Error fetching {podcast_name}: {e}")
            self.log_operation('fetch_episodes', podcast_name, '', 'error', str(e))
            return []
    
    def extract_episode_url(self, entry) -> Optional[str]:
        """Extract the main episode page URL (not audio file)"""
        # Priority order for finding episode URLs:
        
        # 1. Try the main link (usually the episode page)
        if hasattr(entry, 'link') and entry.link:
            return entry.link
        
        # 2. Try links array for non-audio links
        if hasattr(entry, 'links'):
            for link in entry.links:
                link_type = link.get('type', '').lower()
                # Skip audio files, look for HTML pages
                if link_type in ['text/html', 'application/xhtml+xml', ''] or 'audio' not in link_type:
                    if link.get('href'):
                        return link.get('href')
        
        # 3. Try guid if it's a URL
        if hasattr(entry, 'guid'):
            guid = entry.guid
            if isinstance(guid, str) and (guid.startswith('http') or guid.startswith('www')):
                return guid
        
        # 4. Try id field
        if hasattr(entry, 'id'):
            entry_id = entry.id
            if isinstance(entry_id, str) and (entry_id.startswith('http') or entry_id.startswith('www')):
                return entry_id
        
        return None
    
    def extract_audio_url(self, entry) -> Optional[str]:
        """Extract audio URL from RSS entry"""
        # Look for audio enclosures
        if hasattr(entry, 'enclosures'):
            for enclosure in entry.enclosures:
                if 'audio' in enclosure.get('type', ''):
                    return enclosure.get('href')
        
        # Fallback to links
        if hasattr(entry, 'links'):
            for link in entry.links:
                if 'audio' in link.get('type', ''):
                    return link.get('href')
        
        return None
    
    def extract_duration(self, entry) -> Optional[int]:
        """Extract episode duration in minutes"""
        # Try to extract duration from various fields
        duration_str = getattr(entry, 'itunes_duration', '') or getattr(entry, 'duration', '')
        
        if duration_str:
            try:
                # Parse duration (format: HH:MM:SS or MM:SS)
                parts = duration_str.split(':')
                if len(parts) == 3:  # HH:MM:SS
                    return int(parts[0]) * 60 + int(parts[1])
                elif len(parts) == 2:  # MM:SS
                    return int(parts[0])
            except:
                pass
        
        return None
    
    def extract_intelligence(self, transcript: str, podcast_name: str, episode_title: str, podcast_config: Dict) -> Optional[Dict]:
        """Enhanced intelligence extraction with executive communication principles"""
        if not self.anthropic_api_key or not transcript:
            self.logger.warning(f"Missing API key or transcript for {episode_title}")
            return None
        
        try:
            focus = podcast_config.get('focus', 'general')
            priority = podcast_config.get('priority', 'medium')
            
            # Get focus-specific extraction guidance
            focus_config = self.config.get('focus_areas', {}).get(focus, {})
            extraction_emphasis = focus_config.get('extraction_emphasis', "Focus on key insights and actionable information")
            
            # Enhanced prompt with executive communication principles
            prompt = f"""
            As a senior AI industry analyst, analyze this {podcast_name} episode and create an executive-level intelligence brief.
            
            PODCAST CONTEXT:
            - Name: {podcast_name}
            - Focus Area: {focus}
            - Extraction Emphasis: {extraction_emphasis}
            - Episode: {episode_title}
            
            TRANSCRIPT CONTENT:
            {transcript[:4500]}
            
            Create a comprehensive analysis following executive communication best practices:
            
            1. START WITH HEADLINE: Lead with the most important takeaway
            2. DESIGN FOR SKIMMING: Use clear structure and bold key points  
            3. ANSWER OBJECTIONS: Address potential concerns upfront
            4. SHOW IMPACT: Quantify business implications where possible
            
            Provide your analysis in this exact JSON format (NO markdown, NO code blocks, just clean JSON):
            {{
                "headline_takeaway": "One powerful sentence capturing the most important insight from this episode",
                "executive_summary": "A comprehensive 4-6 sentence summary that starts with the headline version, then provides crucial context. Focus on business impact, strategic implications, and actionable insights. Write for executives who scan quickly - each sentence should add clear value.",
                "strategic_implications": ["3-4 high-level business or industry implications that executives should understand"],
                "technical_developments": ["Specific technical advances, tools, frameworks, or breakthroughs mentioned with business context"],
                "market_dynamics": ["Business trends, competitive insights, market shifts, or economic implications discussed"],
                "key_people": ["Notable people mentioned in format: 'Name (Role at Company) - Key contribution or quote'"],
                "companies_mentioned": ["Companies discussed with specific context: 'Company Name - What was said about them'"],
                "predictions": ["Future predictions with timelines and confidence indicators where mentioned"],
                "actionable_insights": ["4-6 specific, implementable takeaways that listeners can act on"],
                "risk_factors": ["Potential challenges, risks, or concerns mentioned that could impact strategy"],
                "quantified_impact": ["Any specific numbers, percentages, timelines, or measurable outcomes mentioned"],
                "bottom_line": "One sentence that captures the core message: what should executives remember from this episode?",
                "importance_score": 8,
                "confidence_score": 0.9,
                "guest_expertise": "Brief description of the main speaker's background and why their perspective matters"
            }}
            
            CRITICAL REQUIREMENTS:
            - Executive summary must be 4-6 substantive sentences minimum
            - Focus on business impact and strategic value
            - Use clear, decisive language
            - Quantify impact where possible
            - Address potential objections or concerns
            - Return clean JSON only - no formatting, no code blocks, no extra text
            """
            
            start_time = datetime.now()
            
            # Call Claude API with enhanced settings
            response = requests.post(
                "https://api.anthropic.com/v1/messages",
                headers={
                    "Content-Type": "application/json",
                    "x-api-key": self.anthropic_api_key,
                    "anthropic-version": "2023-06-01"
                },
                json={
                    "model": "claude-sonnet-4-20250514",
                    "max_tokens": 3500,
                    "temperature": 0.2,
                    "messages": [{"role": "user", "content": prompt}]
                }
            )
            
            processing_time = (datetime.now() - start_time).total_seconds()
            
            if response.status_code == 200:
                result = response.json()
                claude_response = result['content'][0]['text'].strip()
                
                # Clean up any markdown formatting that might appear
                if claude_response.startswith('```json'):
                    claude_response = claude_response.replace('```json', '').replace('```', '').strip()
                if claude_response.startswith('```'):
                    claude_response = claude_response.replace('```', '').strip()
                
                # Calculate processing cost
                input_tokens = len(prompt) / 4
                output_tokens = len(claude_response) / 4
                cost = (input_tokens * 3 + output_tokens * 15) / 1_000_000
                
                try:
                    # Parse JSON response
                    intelligence_data = json.loads(claude_response)
                    
                    # Add metadata
                    intelligence_data.update({
                        'processing_cost': cost,
                        'processing_time_seconds': processing_time,
                        'model_used': 'claude-sonnet-4-20250514',
                        'extraction_timestamp': datetime.now().isoformat()
                    })
                    
                    self.logger.info(f"Successfully extracted enhanced intelligence from {episode_title} (${cost:.4f}, {processing_time:.1f}s)")
                    self.log_operation('extract_intelligence', podcast_name, episode_title, 'success', cost=cost, processing_time=processing_time)
                    
                    return intelligence_data
                    
                except json.JSONDecodeError as e:
                    self.logger.warning(f"Failed to parse JSON response for {episode_title}: {e}")
                    self.logger.debug(f"Raw response: {claude_response[:200]}...")
                    
                    # Fallback response with executive summary
                    fallback_data = {
                        'headline_takeaway': f"Analysis of {episode_title} from {podcast_name}",
                        'executive_summary': claude_response[:800] + "..." if len(claude_response) > 800 else claude_response,
                        'processing_cost': cost,
                        'processing_time_seconds': processing_time,
                        'model_used': 'claude-sonnet-4-20250514',
                        'importance_score': 5,
                        'confidence_score': 0.5,
                        'bottom_line': 'Raw analysis available in executive summary due to parsing issue',
                        'parsing_error': True
                    }
                    
                    return fallback_data
                    
            else:
                error_msg = f"Claude API error: {response.status_code} - {response.text}"
                self.logger.error(error_msg)
                self.log_operation('extract_intelligence', podcast_name, episode_title, 'error', error_message=error_msg)
                return None
                
        except Exception as e:
            error_msg = f"Intelligence extraction error: {e}"
            self.logger.error(error_msg)
            self.log_operation('extract_intelligence', podcast_name, episode_title, 'error', error_message=error_msg)
            return None
    
    def save_episode(self, episode_data: Dict) -> Optional[int]:
        """Save episode to database with enhanced metadata"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        try:
            cursor.execute('''
                INSERT OR IGNORE INTO episodes 
                (podcast_name, title, pub_date, description, audio_url, episode_url, duration_minutes, guid)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                episode_data['podcast_name'],
                episode_data['title'],
                episode_data['pub_date'],
                episode_data['description'],
                episode_data.get('audio_url'),
                episode_data.get('episode_url'),
                episode_data.get('duration_minutes'),
                episode_data.get('guid')
            ))
            
            # Get episode ID
            cursor.execute('''
                SELECT id FROM episodes WHERE podcast_name = ? AND title = ?
            ''', (episode_data['podcast_name'], episode_data['title']))
            
            result = cursor.fetchone()
            episode_id = result[0] if result else None
            
            conn.commit()
            conn.close()
            
            if episode_id:
                self.logger.debug(f"Saved episode: {episode_data['title']}")
            
            return episode_id
            
        except Exception as e:
            self.logger.error(f"Error saving episode: {e}")
            conn.close()
            return None
    
    # Recreate the enhanced save_intelligence method
    def save_intelligence(self, episode_id: int, intelligence_data: Dict):
        """Save intelligence data, replacing any existing record for this episode"""
        import json
        import sqlite3
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        try:
            # Check if intelligence already exists for this episode
            cursor.execute('SELECT id FROM intelligence WHERE episode_id = ?', (episode_id,))
            existing = cursor.fetchone()
            
            if existing:
                intelligence_id = existing[0]
                self.logger.info(f"⚠️ Intelligence already exists for episode {episode_id}, updating...")
                
                # Update existing record
                cursor.execute('''
                    UPDATE intelligence SET
                        headline_takeaway = ?, executive_summary = ?, strategic_implications = ?,
                        technical_developments = ?, market_dynamics = ?, key_people = ?, 
                        companies_mentioned = ?, predictions = ?, actionable_insights = ?,
                        risk_factors = ?, quantified_impact = ?, bottom_line = ?, 
                        guest_expertise = ?, importance_score = ?, confidence_score = ?, 
                        processing_cost = ?, processing_time_seconds = ?, model_used = ?,
                        episode_url = ?, processed_at = CURRENT_TIMESTAMP
                    WHERE episode_id = ?
                ''', (
                    intelligence_data.get('headline_takeaway', ''),
                    intelligence_data.get('executive_summary', ''),
                    json.dumps(intelligence_data.get('strategic_implications', [])),
                    json.dumps(intelligence_data.get('technical_developments', [])),
                    json.dumps(intelligence_data.get('market_dynamics', [])),
                    json.dumps(intelligence_data.get('key_people', [])),
                    json.dumps(intelligence_data.get('companies_mentioned', [])),
                    json.dumps(intelligence_data.get('predictions', [])),
                    json.dumps(intelligence_data.get('actionable_insights', [])),
                    json.dumps(intelligence_data.get('risk_factors', [])),
                    json.dumps(intelligence_data.get('quantified_impact', [])),
                    intelligence_data.get('bottom_line', ''),
                    intelligence_data.get('guest_expertise', ''),
                    int(intelligence_data.get('importance_score', 5)),
                    float(intelligence_data.get('confidence_score', 0.5)),
                    intelligence_data.get('processing_cost', 0.0),
                    intelligence_data.get('processing_time_seconds', 0.0),
                    intelligence_data.get('model_used', 'unknown'),
                    intelligence_data.get('episode_url', ''),
                    episode_id
                ))
            else:
                # Insert new record
                cursor.execute('''
                    INSERT INTO intelligence (
                        episode_id, headline_takeaway, executive_summary, strategic_implications,
                        technical_developments, market_dynamics, key_people, companies_mentioned,
                        predictions, actionable_insights, risk_factors, quantified_impact,
                        bottom_line, guest_expertise, importance_score, confidence_score, 
                        processing_cost, processing_time_seconds, model_used, episode_url
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    episode_id,
                    intelligence_data.get('headline_takeaway', ''),
                    intelligence_data.get('executive_summary', ''),
                    json.dumps(intelligence_data.get('strategic_implications', [])),
                    json.dumps(intelligence_data.get('technical_developments', [])),
                    json.dumps(intelligence_data.get('market_dynamics', [])),
                    json.dumps(intelligence_data.get('key_people', [])),
                    json.dumps(intelligence_data.get('companies_mentioned', [])),
                    json.dumps(intelligence_data.get('predictions', [])),
                    json.dumps(intelligence_data.get('actionable_insights', [])),
                    json.dumps(intelligence_data.get('risk_factors', [])),
                    json.dumps(intelligence_data.get('quantified_impact', [])),
                    intelligence_data.get('bottom_line', ''),
                    intelligence_data.get('guest_expertise', ''),
                    int(intelligence_data.get('importance_score', 5)),
                    float(intelligence_data.get('confidence_score', 0.5)),
                    intelligence_data.get('processing_cost', 0.0),
                    intelligence_data.get('processing_time_seconds', 0.0),
                    intelligence_data.get('model_used', 'unknown'),
                    intelligence_data.get('episode_url', '')
                ))
            
            # Mark episode as processed
            cursor.execute('UPDATE episodes SET processed = 1, updated_at = CURRENT_TIMESTAMP WHERE id = ?', (episode_id,))
            
            conn.commit()
            self.logger.debug(f"Intelligence saved for episode ID: {episode_id}")
            
        except Exception as e:
            self.logger.error(f"Error saving intelligence: {e}")
            conn.rollback()
        finally:
            conn.close()

    
    def log_operation(self, operation_type: str, podcast_name: str, episode_title: str, 
                     status: str, error_message: str = None, cost: float = 0.0, processing_time: float = 0.0):
        """Log system operations for monitoring"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        try:
            cursor.execute('''
                INSERT INTO processing_logs 
                (operation_type, podcast_name, episode_title, status, error_message, cost, processing_time)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (operation_type, podcast_name, episode_title, status, error_message, cost, processing_time))
            
            conn.commit()
            conn.close()
            
        except Exception as e:
            self.logger.error(f"Error logging operation: {e}")
            if conn:
                conn.close()
    
    def check_cost_limits(self, estimated_cost: float) -> bool:
        """Check if processing would exceed cost limits"""
        today = datetime.now().date()
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Get today's costs
        cursor.execute('''
            SELECT COALESCE(SUM(processing_cost), 0) FROM intelligence 
            WHERE DATE(processed_at) = ?
        ''', (today,))
        daily_cost = cursor.fetchone()[0]
        
        # Get this week's costs
        week_start = today - timedelta(days=today.weekday())
        cursor.execute('''
            SELECT COALESCE(SUM(processing_cost), 0) FROM intelligence 
            WHERE DATE(processed_at) >= ?
        ''', (week_start,))
        weekly_cost = cursor.fetchone()[0]
        
        conn.close()
        
        # Check limits
        daily_limit = self.config['cost_limits']['daily_max_usd']
        weekly_limit = self.config['cost_limits']['weekly_max_usd']
        
        if daily_cost + estimated_cost > daily_limit:
            self.logger.warning(f"Daily cost limit would be exceeded: ${daily_cost + estimated_cost:.4f} > ${daily_limit}")
            return False
        
        if weekly_cost + estimated_cost > weekly_limit:
            self.logger.warning(f"Weekly cost limit would be exceeded: ${weekly_cost + estimated_cost:.4f} > ${weekly_limit}")
            return False
        
        return True
    
    def process_all_podcasts(self) -> Dict:
        """Enhanced podcast processing with comprehensive monitoring"""
        self.logger.info("=== Starting enhanced podcast processing batch ===")
        start_time = datetime.now()
        
        active_podcasts = self.get_active_podcasts()
        estimated_cost = len(active_podcasts) * 0.05  # Rough estimate
        
        self.logger.info(f"Processing {len(active_podcasts)} active podcasts")
        self.logger.info(f"Estimated cost: ${estimated_cost:.4f}")
        
        # Check cost limits
        if not self.check_cost_limits(estimated_cost):
            self.logger.error("Processing would exceed cost limits - aborting")
            return {'error': 'Cost limits exceeded', 'processed_episodes': 0}
        
        results = {
            'processed_episodes': 0,
            'failed_episodes': 0,
            'skipped_episodes': 0,
            'total_cost': 0.0,
            'total_processing_time': 0.0,
            'podcasts_processed': [],
            'podcasts_failed': [],
            'episode_details': [],
            'errors': []
        }
        
        for podcast_name, podcast_config in active_podcasts.items():
            podcast_start_time = datetime.now()
            
            try:
                self.logger.info(f"Processing {podcast_name} (priority: {podcast_config.get('priority', 'medium')})")
                
                # Fetch recent episodes with enhanced URL extraction
                episodes = self.fetch_recent_episodes(podcast_name, podcast_config)
                
                if not episodes:
                    self.logger.info(f"No new episodes found for {podcast_name}")
                    continue
                
                podcast_episodes_processed = 0
                
                for episode_data in episodes:
                    try:
                        # Check if episode already exists and is processed
                        episode_id = self.save_episode(episode_data)
                        
                        if episode_id and episode_data.get('description'):
                            # Check if already processed
                            conn = sqlite3.connect(self.db_path)
                            cursor = conn.cursor()
                            cursor.execute('SELECT processed FROM episodes WHERE id = ?', (episode_id,))
                            already_processed = cursor.fetchone()[0]
                            conn.close()
                            
                            if already_processed:
                                self.logger.debug(f"Episode already processed: {episode_data['title']}")
                                results['skipped_episodes'] += 1
                                continue
                            
                            # Extract intelligence with enhanced prompting
                            intelligence = self.extract_intelligence(
                                episode_data['description'],
                                podcast_name,
                                episode_data['title'],
                                podcast_config
                            )
                            
                            if intelligence:
                                # Add episode URL to intelligence data
                                intelligence['episode_url'] = episode_data.get('episode_url', '')
                                
                                self.save_intelligence(episode_id, intelligence)
                                results['processed_episodes'] += 1
                                podcast_episodes_processed += 1
                                results['total_cost'] += intelligence.get('processing_cost', 0)
                                results['total_processing_time'] += intelligence.get('processing_time_seconds', 0)
                                
                                # Store episode details for reporting
                                results['episode_details'].append({
                                    'podcast': podcast_name,
                                    'title': episode_data['title'],
                                    'summary': intelligence.get('executive_summary', ''),
                                    'headline': intelligence.get('headline_takeaway', ''),
                                    'importance': intelligence.get('importance_score', 5),
                                    'confidence': intelligence.get('confidence_score', 0.5),
                                    'cost': intelligence.get('processing_cost', 0),
                                    'pub_date': episode_data['pub_date'],
                                    'episode_url': episode_data.get('episode_url')
                                })
                                
                                self.logger.info(f"✅ Processed: {episode_data['title'][:50]}...")
                                
                            else:
                                results['failed_episodes'] += 1
                                self.logger.warning(f"Failed to extract intelligence from: {episode_data['title']}")
                        
                        else:
                            results['skipped_episodes'] += 1
                            self.logger.debug(f"Skipped episode (no description): {episode_data['title']}")
                    
                    except Exception as e:
                        results['failed_episodes'] += 1
                        error_msg = f"Error processing episode {episode_data.get('title', 'unknown')}: {e}"
                        self.logger.error(error_msg)
                        results['errors'].append(error_msg)
                
                if podcast_episodes_processed > 0:
                    results['podcasts_processed'].append(podcast_name)
                    podcast_time = (datetime.now() - podcast_start_time).total_seconds()
                    self.logger.info(f"✅ {podcast_name}: {podcast_episodes_processed} episodes processed in {podcast_time:.1f}s")
                else:
                    self.logger.info(f"⚠️ {podcast_name}: No episodes processed")
                
            except Exception as e:
                results['podcasts_failed'].append(podcast_name)
                error_msg = f"Error processing podcast {podcast_name}: {e}"
                self.logger.error(error_msg)
                results['errors'].append(error_msg)
        
        # Calculate total processing time
        total_time = (datetime.now() - start_time).total_seconds()
        results['batch_processing_time'] = total_time
        
        # Log batch summary
        self.logger.info("=== Enhanced podcast processing batch complete ===")
        self.logger.info(f"Total time: {total_time:.1f}s")
        self.logger.info(f"Episodes processed: {results['processed_episodes']}")
        self.logger.info(f"Episodes failed: {results['failed_episodes']}")
        self.logger.info(f"Episodes skipped: {results['skipped_episodes']}")
        self.logger.info(f"Total cost: ${results['total_cost']:.4f}")
        self.logger.info(f"Podcasts successful: {len(results['podcasts_processed'])}")
        
        return results
    
    def generate_enhanced_report(self) -> Tuple[str, str]:
        """Generate enhanced executive-level intelligence report"""
        self.logger.info("Generating enhanced intelligence reports...")
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Get recent episodes with enhanced fields
        days_back = 7
        since_date = (datetime.now() - timedelta(days=days_back)).isoformat()
        
        cursor.execute('''
            SELECT e.podcast_name, e.title, e.pub_date, e.duration_minutes, e.episode_url,
                   i.headline_takeaway, i.executive_summary, i.strategic_implications,
                   i.technical_developments, i.market_dynamics, i.key_people, 
                   i.companies_mentioned, i.predictions, i.actionable_insights,
                   i.importance_score, i.confidence_score, i.processing_cost, i.processed_at
            FROM episodes e
            JOIN intelligence i ON e.id = i.episode_id
            WHERE i.processed_at >= ?
            ORDER BY i.importance_score DESC, i.processed_at DESC
        ''', (since_date,))
        
        episodes = cursor.fetchall()
        
        # Get statistics
        cursor.execute('''
            SELECT 
                COUNT(*) as episode_count,
                SUM(processing_cost) as total_cost,
                AVG(importance_score) as avg_importance,
                AVG(confidence_score) as avg_confidence,
                COUNT(DISTINCT podcast_name) as podcast_count,
                SUM(CASE WHEN importance_score >= 8 THEN 1 ELSE 0 END) as high_impact_count
            FROM episodes e
            JOIN intelligence i ON e.id = i.episode_id
            WHERE i.processed_at >= ?
        ''', (since_date,))
        
        stats = cursor.fetchone()
        conn.close()
        
        if not episodes:
            no_data_msg = f"No episodes processed in the last {days_back} days"
            return no_data_msg, no_data_msg
        
        # Process data for report
        all_companies = []
        all_people = []
        all_insights = []
        headlines = []
        
        for row in episodes:
            try:
                # Extract headlines
                if row[5]:  # headline_takeaway
                    headlines.append(row[5])
                
                # Parse JSON fields
                if row[10]: all_people.extend(json.loads(row[10]))
                if row[11]: all_companies.extend(json.loads(row[11]))
                if row[13]: all_insights.extend(json.loads(row[13]))
                
            except json.JSONDecodeError:
                continue
        
        # Get top items
        from collections import Counter
        top_companies = Counter(all_companies).most_common(8)
        top_people = Counter(all_people).most_common(8)
        top_insights = list(set(all_insights))[:6]
        
        # Generate report metadata
        report_date = datetime.now().strftime("%B %d, %Y at %I:%M %p")
        week_range = f"{(datetime.now() - timedelta(days=days_back)).strftime('%B %d')} - {datetime.now().strftime('%B %d, %Y')}"
        
        # Create enhanced HTML report
        html_report = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>AI Intelligence Executive Brief - {report_date}</title>
            <style>
                body {{
                    font-family: 'Segoe UI', -apple-system, BlinkMacSystemFont, Roboto, sans-serif;
                    line-height: 1.6; color: #2c3e50; max-width: 1200px; margin: 0 auto;
                    padding: 20px; background-color: #f8f9fa;
                }}
                .header {{
                    background: linear-gradient(135deg, #2c3e50 0%, #34495e 100%);
                    color: white; padding: 40px 30px; border-radius: 15px; text-align: center;
                    margin-bottom: 30px; box-shadow: 0 10px 30px rgba(0,0,0,0.15);
                }}
                .executive-summary {{
                    background: white; border-left: 6px solid #e74c3c; padding: 30px;
                    margin: 30px 0; border-radius: 10px; box-shadow: 0 6px 20px rgba(0,0,0,0.1);
                }}
                .stats-grid {{
                    display: grid; grid-template-columns: repeat(auto-fit, minmax(180px, 1fr));
                    gap: 20px; margin: 30px 0;
                }}
                .stat-card {{
                    background: white; padding: 25px; border-radius: 12px; text-align: center;
                    box-shadow: 0 4px 15px rgba(0,0,0,0.1); border-top: 4px solid #3498db;
                }}
                .section {{
                    background: white; margin: 30px 0; padding: 35px; border-radius: 12px;
                    box-shadow: 0 4px 15px rgba(0,0,0,0.1);
                }}
                .episode-card {{
                    background: #f8f9fa; margin: 25px 0; padding: 30px; border-radius: 12px;
                    border-left: 6px solid #3498db;
                }}
                .tag {{
                    background: linear-gradient(135deg, #3498db, #2980b9); color: white;
                    padding: 8px 14px; border-radius: 20px; font-size: 0.9em; margin: 3px;
                    display: inline-block;
                }}
                .episode-link {{ margin-top: 15px; padding-top: 15px; border-top: 1px solid #ecf0f1; }}
                .episode-link a {{ color: #3498db; text-decoration: none; font-weight: 500; }}
            </style>
        </head>
        <body>
            <div class="header">
                <h1>AI Intelligence Executive Brief</h1>
                <p>Week of {week_range}</p>
                <p>Generated on {report_date}</p>
            </div>
            
            <div class="executive-summary">
                <h2>Executive Summary</h2>
                <p><strong>Bottom line:</strong> This week's AI intelligence reveals {stats[5] or 0} high-impact developments across {stats[4] or 0} key podcasts.</p>
                <h3>This Week's Headlines</h3>
                <ul>
        """
        
        for headline in headlines[:5]:
            if headline:
                html_report += f"<li><strong>{headline}</strong></li>\n"
        
        html_report += f"""
                </ul>
            </div>
            
            <div class="stats-grid">
                <div class="stat-card"><h3>{stats[0] or 0}</h3><p>Episodes</p></div>
                <div class="stat-card"><h3>{stats[5] or 0}</h3><p>High-Impact</p></div>
                <div class="stat-card"><h3>${stats[1] or 0:.3f}</h3><p>Cost</p></div>
                <div class="stat-card"><h3>{stats[2] or 0:.1f}/10</h3><p>Avg Importance</p></div>
            </div>
            
            <div class="section">
                <h2>Companies Mentioned</h2>
        """
        
        for company, count in top_companies:
            if company:
                html_report += f'<span class="tag">{company} ({count})</span>\n'
        
        html_report += f"""
            </div>
            
            <div class="section">
                <h2>Episode Intelligence</h2>
        """
        
        for i, episode in enumerate(episodes[:10], 1):
            (podcast_name, title, pub_date, duration, episode_url, headline, summary,
             _, _, _, _, _, _, _, importance, confidence, cost, _) = episode
            
            html_report += f"""
            <div class="episode-card">
                <h3>{i}. {podcast_name}</h3>
                <h4>{title}</h4>
                <p><strong>Headline:</strong> {headline}</p>
                <p>{summary}</p>
                <p><strong>Impact:</strong> {importance}/10 | <strong>Cost:</strong> ${cost:.4f}</p>
                {f'<div class="episode-link"><a href="{episode_url}" target="_blank">Listen to Episode</a></div>' if episode_url else ''}
            </div>
            """
        
        html_report += """
            </div>
        </body>
        </html>
        """
        
        # Simple markdown version
        markdown_report = f"""# AI Intelligence Executive Brief
## Week of {week_range}

### Statistics
- Episodes: {stats[0] or 0}
- High-Impact: {stats[5] or 0}
- Cost: ${stats[1] or 0:.4f}

### Headlines
"""
        for headline in headlines[:5]:
            if headline:
                markdown_report += f"- {headline}\n"
        
        return html_report, markdown_report
    
    def save_and_display_reports(self, html_report: str, markdown_report: str) -> Tuple[str, str]:
        """Save reports to files"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        html_filename = self.reports_dir / f"AI_Intelligence_Report_{timestamp}.html"
        markdown_filename = self.reports_dir / f"AI_Intelligence_Report_{timestamp}.md"
        
        with open(html_filename, 'w', encoding='utf-8') as f:
            f.write(html_report)
        
        with open(markdown_filename, 'w', encoding='utf-8') as f:
            f.write(markdown_report)
        
        self.logger.info(f"Reports saved: {html_filename.name}")
        return str(html_filename), str(markdown_filename)
    
    def get_system_statistics(self) -> Dict:
        """Get comprehensive system statistics"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('SELECT COUNT(*) FROM episodes')
        total_episodes = cursor.fetchone()[0]
        
        cursor.execute('SELECT COUNT(*) FROM intelligence')
        processed_episodes = cursor.fetchone()[0]
        
        cursor.execute('SELECT COALESCE(SUM(processing_cost), 0) FROM intelligence')
        total_cost = cursor.fetchone()[0]
        
        cursor.execute('SELECT COALESCE(AVG(importance_score), 0) FROM intelligence')
        avg_importance = cursor.fetchone()[0]
        
        week_ago = (datetime.now() - timedelta(days=7)).isoformat()
        cursor.execute('SELECT COUNT(*) FROM intelligence WHERE processed_at >= ?', (week_ago,))
        recent_processed = cursor.fetchone()[0]
        
        conn.close()
        
        return {
            'total_episodes': total_episodes,
            'processed_episodes': processed_episodes,
            'processing_rate': f"{(processed_episodes/max(total_episodes, 1)*100):.1f}%",
            'total_cost': total_cost,
            'avg_importance': avg_importance,
            'recent_processed': recent_processed
        }

# Initialize the production system
if config and anthropic_api_key:
    print("\n🔧 INITIALIZING PRODUCTION SYSTEM...")
    system = ProductionPodcastIntelligence(config)
    print("✅ Production system ready!")
    
    stats = system.get_system_statistics()
    print(f"\n📊 SYSTEM OVERVIEW:")
    print(f"📻 Total episodes: {stats['total_episodes']}")
    print(f"🧠 Processed: {stats['processed_episodes']} ({stats['processing_rate']})")
    print(f"💰 Total cost: ${stats['total_cost']:.4f}")
    print(f"⭐ Avg importance: {stats['avg_importance']:.1f}/10")
    print(f"📈 Recent: {stats['recent_processed']} episodes this week")
else:
    print("❌ System initialization failed - check configuration and API key")
    system = None

    
# Apply the fix
system.save_intelligence = save_intelligence_no_duplicates.__get__(system, ProductionPodcastIntelligence)
print("✅ Enhanced save_intelligence method applied!")

✅ Enhanced save_intelligence method applied successfully!

🔧 INITIALIZING PRODUCTION SYSTEM...
2025-09-23 20:54:57,262 - PodcastIntelligence - INFO - Database setup completed
2025-09-23 20:54:57,264 - PodcastIntelligence - INFO - Production system initialized with 9 active podcasts
✅ Production system ready!

📊 SYSTEM OVERVIEW:
📻 Total episodes: 8
🧠 Processed: 9 (112.5%)
💰 Total cost: $0.2084
⭐ Avg importance: 6.9/10
📈 Recent: 6 episodes this week


NameError: name 'save_intelligence_no_duplicates' is not defined

In [10]:
# CELL 3.5: Enhanced Methods Update
# Update existing methods with enhanced functionality

# Replace the extract_intelligence method
def enhanced_extract_intelligence(self, transcript: str, podcast_name: str, episode_title: str, podcast_config: Dict) -> Optional[Dict]:
    """Extract AI intelligence with enhanced executive-focused prompting"""
    if not self.anthropic_api_key or not transcript:
        self.logger.warning(f"Missing API key or transcript for {episode_title}")
        return None
    
    try:
        focus = podcast_config.get('focus', 'general')
        priority = podcast_config.get('priority', 'medium')
        
        # Get focus-specific extraction guidance
        focus_config = self.config.get('focus_areas', {}).get(focus, {})
        extraction_emphasis = focus_config.get('extraction_emphasis', "Focus on key insights and actionable information")
        
        # ENHANCED PROMPT with executive communication principles
        prompt = f"""
        As a senior AI industry analyst, analyze this {podcast_name} episode and create an executive-level intelligence brief.
        
        PODCAST CONTEXT:
        - Name: {podcast_name}
        - Focus Area: {focus}
        - Extraction Emphasis: {extraction_emphasis}
        - Episode: {episode_title}
        
        TRANSCRIPT CONTENT:
        {transcript[:4500]}
        
        Create a comprehensive analysis following executive communication best practices:
        
        1. START WITH HEADLINE: Lead with the most important takeaway
        2. DESIGN FOR SKIMMING: Use clear structure and bold key points  
        3. ANSWER OBJECTIONS: Address potential concerns upfront
        4. SHOW IMPACT: Quantify business implications where possible
        
        Provide your analysis in this exact JSON format (NO markdown, NO code blocks, just clean JSON):
        {{
            "headline_takeaway": "One powerful sentence capturing the most important insight from this episode",
            "executive_summary": "A comprehensive 4-6 sentence summary that starts with the headline version, then provides crucial context. Focus on business impact, strategic implications, and actionable insights. Write for executives who scan quickly - each sentence should add clear value.",
            "strategic_implications": ["3-4 high-level business or industry implications that executives should understand"],
            "technical_developments": ["Specific technical advances, tools, frameworks, or breakthroughs mentioned with business context"],
            "market_dynamics": ["Business trends, competitive insights, market shifts, or economic implications discussed"],
            "key_people": ["Notable people mentioned in format: 'Name (Role at Company) - Key contribution or quote'"],
            "companies_mentioned": ["Companies discussed with specific context: 'Company Name - What was said about them'"],
            "predictions": ["Future predictions with timelines and confidence indicators where mentioned"],
            "actionable_insights": ["4-6 specific, implementable takeaways that listeners can act on"],
            "risk_factors": ["Potential challenges, risks, or concerns mentioned that could impact strategy"],
            "quantified_impact": ["Any specific numbers, percentages, timelines, or measurable outcomes mentioned"],
            "bottom_line": "One sentence that captures the core message: what should executives remember from this episode?",
            "importance_score": 8,
        
            "episode_url": "",
            "guest_expertise": "Brief description of the main speaker's background and why their perspective matters"
        }}
        
        CRITICAL REQUIREMENTS:
        - Executive summary must be 4-6 substantive sentences minimum
        - Focus on business impact and strategic value
        - Use clear, decisive language
        - Quantify impact where possible
        - Address potential objections or concerns
        - Return clean JSON only - no formatting, no code blocks, no extra text
        """
        
        start_time = datetime.now()
        
        # Call Claude API with enhanced settings
        response = requests.post(
            "https://api.anthropic.com/v1/messages",
            headers={
                "Content-Type": "application/json",
                "x-api-key": self.anthropic_api_key,
                "anthropic-version": "2023-06-01"
            },
            json={
                "model": "claude-sonnet-4-20250514",
                "max_tokens": 3500,  # Increased for longer summaries
                "temperature": 0.2,   # Lower for more focused output
                "messages": [{"role": "user", "content": prompt}]
            }
        )
        
        processing_time = (datetime.now() - start_time).total_seconds()
        
        if response.status_code == 200:
            result = response.json()
            claude_response = result['content'][0]['text'].strip()
            
            # Clean up any markdown formatting that might appear
            if claude_response.startswith('```json'):
                claude_response = claude_response.replace('```json', '').replace('```', '').strip()
            if claude_response.startswith('```'):
                claude_response = claude_response.replace('```', '').strip()
            
            # Calculate processing cost
            input_tokens = len(prompt) / 4
            output_tokens = len(claude_response) / 4
            cost = (input_tokens * 3 + output_tokens * 15) / 1_000_000
            
            try:
                # Parse JSON response
                intelligence_data = json.loads(claude_response)
                
                # Add metadata
                intelligence_data.update({
                    'processing_cost': cost,
                    'processing_time_seconds': processing_time,
                    'model_used': 'claude-sonnet-4-20250514',
                    'extraction_timestamp': datetime.now().isoformat()
                })
                # Calculate real confidence score
                intelligence_data['confidence_score'] = self.calculate_confidence_score(transcript, intelligence_data, processing_time),
                
                self.logger.info(f"Successfully extracted enhanced intelligence from {episode_title} (${cost:.4f}, {processing_time:.1f}s)")
                self.log_operation('extract_intelligence', podcast_name, episode_title, 'success', cost=cost, processing_time=processing_time)
                
                return intelligence_data
                
            except json.JSONDecodeError as e:
                self.logger.warning(f"Failed to parse JSON response for {episode_title}: {e}")
                self.logger.debug(f"Raw response: {claude_response[:200]}...")
                
                # Fallback response with executive summary
                fallback_data = {
                    'headline_takeaway': f"Analysis of {episode_title} from {podcast_name}",
                    'executive_summary': claude_response[:800] + "..." if len(claude_response) > 800 else claude_response,
                    'processing_cost': cost,
                    'processing_time_seconds': processing_time,
                    'model_used': 'claude-sonnet-4-20250514',
                    'importance_score': 5,
                    'confidence_score': 0.5,
                    'bottom_line': 'Raw analysis available in executive summary due to parsing issue',
                    'parsing_error': True
                }
                
                return fallback_data
                
        else:
            error_msg = f"Claude API error: {response.status_code} - {response.text}"
            self.logger.error(error_msg)
            self.log_operation('extract_intelligence', podcast_name, episode_title, 'error', error_message=error_msg)
            return None
            
    except Exception as e:
        error_msg = f"Intelligence extraction error: {e}"
        self.logger.error(error_msg)
        self.log_operation('extract_intelligence', podcast_name, episode_title, 'error', error_message=error_msg)
        return None

# Replace the save_intelligence method  
def enhanced_save_intelligence(self, episode_id: int, intelligence_data: Dict):
    """Save extracted intelligence with enhanced fields"""
    conn = sqlite3.connect(self.db_path)
    cursor = conn.cursor()
    
    try:
        # First, update the episodes table with episode URL if available
        if intelligence_data.get('episode_url'):
            cursor.execute(
                'UPDATE episodes SET episode_url = ? WHERE id = ?',
                (intelligence_data.get('episode_url'), episode_id)
            )
        
        cursor.execute('''
            INSERT OR REPLACE INTO intelligence (
                episode_id, executive_summary, technical_developments,
                industry_business, key_people, companies_mentioned,
                predictions, key_insights, actionable_items,
                importance_score, confidence_score, processing_cost,
                processing_time_seconds, model_used
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            episode_id,
            # Store the enhanced executive summary
            f"{intelligence_data.get('headline_takeaway', '')} | {intelligence_data.get('executive_summary', '')}",
            json.dumps(intelligence_data.get('technical_developments', [])),
            json.dumps(intelligence_data.get('market_dynamics', intelligence_data.get('industry_business', []))),
            json.dumps(intelligence_data.get('key_people', [])),
            json.dumps(intelligence_data.get('companies_mentioned', [])),
            json.dumps(intelligence_data.get('predictions', [])),
            json.dumps(intelligence_data.get('actionable_insights', intelligence_data.get('key_insights', []))),
            json.dumps(intelligence_data.get('strategic_implications', [])),
            int(intelligence_data.get('importance_score', 5)),
            float(intelligence_data.get('confidence_score', 0.5)),
            intelligence_data.get('processing_cost', 0.0),
            intelligence_data.get('processing_time_seconds', 0.0),
            intelligence_data.get('model_used', 'unknown')
        ))
        
        # Mark episode as processed
        cursor.execute('UPDATE episodes SET processed = 1, updated_at = CURRENT_TIMESTAMP WHERE id = ?', (episode_id,))
        
        conn.commit()
        conn.close()
        
        self.logger.debug(f"Enhanced intelligence saved for episode ID: {episode_id}")
        
    except Exception as e:
        self.logger.error(f"Error saving intelligence: {e}")
        conn.close()


# Replace the generate_production_report method
def enhanced_generate_report(self) -> Tuple[str, str]:
    """Generate executive-level intelligence report with enhanced formatting"""
    self.logger.info("Generating enhanced intelligence reports...")
    
    conn = sqlite3.connect(self.db_path)
    cursor = conn.cursor()
    
    # Get recent episodes with episode URLs
    days_back = 7
    since_date = (datetime.now() - timedelta(days=days_back)).isoformat()
    
    cursor.execute('''
        SELECT e.podcast_name, e.title, e.pub_date, e.duration_minutes, e.episode_url,
               i.executive_summary, i.technical_developments, i.industry_business,
               i.key_people, i.companies_mentioned, i.predictions, i.key_insights,
               i.actionable_items, i.importance_score, i.confidence_score,
               i.processing_cost, i.processed_at
        FROM episodes e
        JOIN intelligence i ON e.id = i.episode_id
        WHERE i.processed_at >= ?
        ORDER BY i.importance_score DESC, i.processed_at DESC
    ''', (since_date,))
    
    episodes = cursor.fetchall()
    
    # Get aggregated statistics
    cursor.execute('''
        SELECT 
            COUNT(*) as episode_count,
            SUM(processing_cost) as total_cost,
            AVG(importance_score) as avg_importance,
            AVG(confidence_score) as avg_confidence,
            COUNT(DISTINCT podcast_name) as podcast_count,
            SUM(CASE WHEN importance_score >= 8 THEN 1 ELSE 0 END) as high_impact_count
        FROM episodes e
        JOIN intelligence i ON e.id = i.episode_id
        WHERE i.processed_at >= ?
    ''', (since_date,))
    
    stats = cursor.fetchone()
    conn.close()
    
    if not episodes:
        no_data_msg = f"📭 No episodes processed in the last {days_back} days"
        return no_data_msg, no_data_msg
    
    # Process aggregated data with enhanced structure
    all_companies = []
    all_people = []
    all_insights = []
    all_strategic = []
    
    for row in episodes:
        try:
            # Parse JSON fields
            companies = json.loads(row[9]) if row[9] else []
            people = json.loads(row[8]) if row[8] else []
            insights = json.loads(row[11]) if row[11] else []
            strategic = json.loads(row[12]) if row[12] else []  # actionable_items
            
            all_companies.extend(companies)
            all_people.extend(people)
            all_insights.extend(insights)
            all_strategic.extend(strategic)
            
        except json.JSONDecodeError:
            continue
    
    # Get top items
    from collections import Counter
    top_companies = Counter(all_companies).most_common(8)
    top_people = Counter(all_people).most_common(8)
    top_insights = list(set(all_insights))[:5]
    top_strategic = list(set(all_strategic))[:6]
    
    # Generate report metadata
    report_date = datetime.now().strftime("%B %d, %Y at %I:%M %p")
    week_range = f"{(datetime.now() - timedelta(days=days_back)).strftime('%B %d')} - {datetime.now().strftime('%B %d, %Y')}"
    
    # Extract top headline takeaways
    headlines = []
    for row in episodes[:5]:  # Top 5 most important episodes
        summary = row[5]  # executive_summary
        if '|' in summary:
            headline = summary.split('|')[0].strip()
            headlines.append(headline)
        else:
            headlines.append(summary[:100] + "..." if len(summary) > 100 else summary)
    
    # Generate Enhanced HTML Report
    html_report = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>AI Intelligence Executive Brief - {report_date}</title>
        <style>
            body {{
                font-family: 'Segoe UI', -apple-system, BlinkMacSystemFont, Roboto, sans-serif;
                line-height: 1.6;
                color: #2c3e50;
                max-width: 1200px;
                margin: 0 auto;
                padding: 20px;
                background-color: #f8f9fa;
            }}
            
            .header {{
                background: linear-gradient(135deg, #2c3e50 0%, #34495e 100%);
                color: white;
                padding: 40px 30px;
                border-radius: 15px;
                text-align: center;
                margin-bottom: 30px;
                box-shadow: 0 10px 30px rgba(0,0,0,0.15);
            }}
            
            .header h1 {{
                margin: 0;
                font-size: 2.5em;
                font-weight: 300;
                letter-spacing: -1px;
            }}
            
            .header .subtitle {{
                margin: 15px 0 5px 0;
                opacity: 0.9;
                font-size: 1.2em;
                font-weight: 500;
            }}
            
            .header .tagline {{
                margin: 5px 0 0 0;
                opacity: 0.8;
                font-size: 1em;
            }}
            
            .executive-summary {{
                background: white;
                border-left: 6px solid #e74c3c;
                padding: 30px;
                margin: 30px 0;
                border-radius: 10px;
                box-shadow: 0 6px 20px rgba(0,0,0,0.1);
            }}
            
            .executive-summary h2 {{
                margin-top: 0;
                color: #e74c3c;
                font-size: 1.8em;
                font-weight: 600;
            }}
            
            .headlines {{
                list-style: none;
                padding: 0;
                margin: 20px 0;
            }}
            
            .headlines li {{
                background: #ecf0f1;
                margin: 12px 0;
                padding: 15px 20px;
                border-radius: 8px;
                border-left: 4px solid #3498db;
                font-weight: 500;
                position: relative;
            }}
            
            .headlines li::before {{
                content: "▶";
                color: #3498db;
                font-weight: bold;
                position: absolute;
                left: 8px;
            }}
            
            .headlines li {{
                padding-left: 35px;
            }}
            
            .stats-grid {{
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(180px, 1fr));
                gap: 20px;
                margin: 30px 0;
            }}
            
            .stat-card {{
                background: white;
                padding: 25px;
                border-radius: 12px;
                text-align: center;
                box-shadow: 0 4px 15px rgba(0,0,0,0.1);
                border-top: 4px solid #3498db;
                transition: transform 0.2s ease;
            }}
            
            .stat-card:hover {{
                transform: translateY(-3px);
            }}
            
            .stat-card h3 {{
                margin: 0 0 10px 0;
                font-size: 2.2em;
                color: #2c3e50;
                font-weight: 700;
            }}
            
            .stat-card p {{
                margin: 0;
                color: #7f8c8d;
                font-weight: 500;
                font-size: 0.95em;
            }}
            
            .section {{
                background: white;
                margin: 30px 0;
                padding: 35px;
                border-radius: 12px;
                box-shadow: 0 4px 15px rgba(0,0,0,0.1);
            }}
            
            .section h2 {{
                color: #2c3e50;
                border-bottom: 3px solid #ecf0f1;
                padding-bottom: 15px;
                margin-top: 0;
                font-size: 1.8em;
                font-weight: 600;
            }}
            
            .episode-card {{
                background: #f8f9fa;
                margin: 25px 0;
                padding: 30px;
                border-radius: 12px;
                border-left: 6px solid #3498db;
                transition: all 0.3s ease;
                position: relative;
            }}
            
            .episode-card:hover {{
                transform: translateY(-2px);
                box-shadow: 0 8px 25px rgba(0,0,0,0.15);
            }}
            
            .episode-header {{
                display: flex;
                justify-content: space-between;
                align-items: flex-start;
                margin-bottom: 20px;
                flex-wrap: wrap;
                gap: 15px;
            }}
            
            .episode-title {{
                flex: 1;
                min-width: 300px;
            }}
            
            .episode-title h3 {{
                margin: 0 0 8px 0;
                color: #2c3e50;
                font-size: 1.4em;
                font-weight: 600;
            }}
            
            .episode-title h4 {{
                margin: 0;
                color: #34495e;
                font-size: 1.1em;
                font-weight: 500;
                line-height: 1.4;
            }}
            
            .episode-meta {{
                display: flex;
                gap: 12px;
                flex-wrap: wrap;
                align-items: center;
            }}
            
            .badge {{
                padding: 6px 14px;
                border-radius: 20px;
                font-size: 0.85em;
                font-weight: 600;
                color: white;
                white-space: nowrap;
            }}
            
            .importance-high {{ background: linear-gradient(135deg, #27ae60, #2ecc71); }}
            .importance-medium {{ background: linear-gradient(135deg, #f39c12, #e67e22); }}
            .importance-low {{ background: linear-gradient(135deg, #95a5a6, #7f8c8d); }}
            
            .confidence-badge {{ background: linear-gradient(135deg, #3498db, #2980b9); }}
            .cost-badge {{ background: linear-gradient(135deg, #9b59b6, #8e44ad); }}
            
            .episode-summary {{
                margin: 20px 0;
                padding: 20px;
                background: white;
                border-radius: 8px;
                border-left: 4px solid #e74c3c;
            }}
            
            .episode-summary .headline {{
                font-weight: 700;
                color: #e74c3c;
                font-size: 1.05em;
                margin-bottom: 10px;
            }}
            
            .episode-summary .details {{
                color: #2c3e50;
                line-height: 1.7;
            }}
            
            .episode-link {{
                margin-top: 15px;
                padding-top: 15px;
                border-top: 1px solid #ecf0f1;
            }}
            
            .episode-link a {{
                color: #3498db;
                text-decoration: none;
                font-weight: 500;
                font-size: 0.95em;
            }}
            
            .episode-link a:hover {{
                text-decoration: underline;
                color: #2980b9;
            }}
            
            .insights-grid {{
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(350px, 1fr));
                gap: 25px;
                margin: 25px 0;
            }}
            
            .insight-box {{
                background: #ecf0f1;
                padding: 20px;
                border-radius: 10px;
                border-left: 4px solid #e74c3c;
                position: relative;
            }}
            
            .insight-box::before {{
                content: "💡";
                position: absolute;
                top: 15px;
                left: 15px;
                font-size: 1.2em;
            }}
            
            .insight-box {{
                padding-left: 50px;
            }}
            
            .companies-people-grid {{
                display: grid;
                grid-template-columns: 1fr 1fr;
                gap: 30px;
                margin: 25px 0;
            }}
            
            .tag-container {{
                display: flex;
                flex-wrap: wrap;
                gap: 8px;
                margin: 15px 0;
            }}
            
            .tag {{
                background: linear-gradient(135deg, #3498db, #2980b9);
                color: white;
                padding: 8px 14px;
                border-radius: 20px;
                font-size: 0.9em;
                font-weight: 500;
                transition: transform 0.2s ease;
            }}
            
            .tag:hover {{
                transform: scale(1.05);
            }}
            
            .bottom-line {{
                background: linear-gradient(135deg, #2c3e50, #34495e);
                color: white;
                padding: 25px 35px;
                border-radius: 12px;
                text-align: center;
                margin: 40px 0;
                box-shadow: 0 6px 20px rgba(0,0,0,0.15);
            }}
            
            .bottom-line h2 {{
                margin: 0 0 15px 0;
                font-size: 1.6em;
                font-weight: 600;
            }}
            
            .bottom-line p {{
                margin: 0;
                font-size: 1.1em;
                opacity: 0.95;
                line-height: 1.6;
            }}
            
            .footer {{
                text-align: center;
                margin: 50px 0 20px 0;
                padding: 30px;
                background: white;
                border-radius: 12px;
                box-shadow: 0 4px 15px rgba(0,0,0,0.1);
                color: #7f8c8d;
            }}
            
            @media (max-width: 768px) {{
                .companies-people-grid {{
                    grid-template-columns: 1fr;
                }}
                
                .episode-header {{
                    flex-direction: column;
                    align-items: stretch;
                }}
                
                .episode-meta {{
                    justify-content: flex-start;
                }}
            }}
        </style>
    </head>
    <body>
        <div class="header">
            <h1>🎯 AI Intelligence Executive Brief</h1>
            <p class="subtitle">Week of {week_range}</p>
            <p class="tagline">Strategic Intelligence • Market Insights • Technical Developments</p>
            <p class="tagline">Generated on {report_date}</p>
        </div>
        
        <div class="executive-summary">
            <h2>📋 Executive Summary</h2>
            <p><strong>Bottom line:</strong> This week's AI intelligence reveals {stats[5] or 0} high-impact developments across {stats[4] or 0} key podcasts, with particularly strong insights in {', '.join([insight[:30] + '...' if len(insight) > 30 else insight for insight in top_insights[:2]])}.</p>
            
            <h3 style="margin: 25px 0 15px 0; color: #2c3e50;">🎯 This Week's Headlines</h3>
            <ul class="headlines">
    """
    
    # Add top headlines
    for headline in headlines:
        if headline:
            html_report += f"<li>{headline}</li>\n"
    
    html_report += f"""
            </ul>
        </div>
        
        <div class="stats-grid">
            <div class="stat-card">
                <h3>{stats[0] or 0}</h3>
                <p>Episodes Analyzed</p>
            </div>
            <div class="stat-card">
                <h3>{stats[5] or 0}</h3>
                <p>High-Impact Stories</p>
            </div>
            <div class="stat-card">
                <h3>{stats[4] or 0}</h3>
                <p>Podcast Sources</p>
            </div>
            <div class="stat-card">
                <h3>{stats[2] or 0:.1f}/10</h3>
                <p>Avg Importance</p>
            </div>
            <div class="stat-card">
                <h3>${stats[1] or 0:.3f}</h3>
                <p>Processing Investment</p>
            </div>
        </div>
        
        <div class="section">
            <h2>💼 Strategic Implications</h2>
            <div class="insights-grid">
    """
    
    for insight in top_strategic:
        if insight:
            html_report += f'<div class="insight-box">{insight}</div>\n'
    
    html_report += f"""
            </div>
        </div>
        
        <div class="section">
            <h2>🏢 Market Intelligence</h2>
            <div class="companies-people-grid">
                <div>
                    <h3 style="color: #2c3e50; margin-bottom: 15px;">Companies in Focus</h3>
                    <div class="tag-container">
    """
    
    for company, count in top_companies:
        if company:
            html_report += f'<span class="tag">{company} ({count})</span>\n'
    
    html_report += f"""
                    </div>
                </div>
                <div>
                    <h3 style="color: #2c3e50; margin-bottom: 15px;">Key Industry Voices</h3>
                    <div class="tag-container">
    """
    
    for person, count in top_people:
        if person:
            # Clean up person format if it includes extra details
            person_clean = person.split(' - ')[0] if ' - ' in person else person
            html_report += f'<span class="tag">{person_clean}</span>\n'
    
    html_report += f"""
                    </div>
                </div>
            </div>
        </div>
        
        <div class="section">
            <h2>📻 Detailed Episode Intelligence</h2>
    """
    
    # Add episode details with enhanced formatting
    for i, episode in enumerate(episodes[:12], 1):  # Show top 12 episodes
        (podcast_name, title, pub_date, duration, episode_url, summary, 
         tech_dev, industry, people, companies, predictions, insights, 
         actionable, importance, confidence, cost, processed_at) = episode
        
        # Determine importance class and text
        if importance >= 8:
            importance_class = "importance-high"
            importance_text = "High Impact"
        elif importance >= 6:
            importance_class = "importance-medium"
            importance_text = "Medium Impact"
        else:
            importance_class = "importance-low"
            importance_text = "Standard"
        
        # Split headline from summary if available
        if '|' in summary:
            headline, details = summary.split('|', 1)
            headline = headline.strip()
            details = details.strip()
        else:
            headline = summary[:100] + "..." if len(summary) > 100 else summary
            details = summary
        
        duration_text = f"{duration} min" if duration else "Duration unknown"
        
        html_report += f"""
        <div class="episode-card">
            <div class="episode-header">
                <div class="episode-title">
                    <h3>{i}. {podcast_name}</h3>
                    <h4>{title}</h4>
                </div>
                
                <div class="episode-meta">
                    <span class="badge {importance_class}">⭐ {importance}/10 • {importance_text}</span>
                    <span class="badge confidence-badge">🎯 {confidence:.1f} Confidence</span>
                    <span class="badge cost-badge">💰 ${cost:.4f}</span>
                </div>
            </div>
            
            <div class="episode-summary">
                <div class="headline">{headline}</div>
                <div class="details">{details}</div>
            </div>
            
            <p style="color: #7f8c8d; margin: 15px 0 0 0; font-size: 0.95em;">
                <strong>Published:</strong> {pub_date[:10]} • <strong>Duration:</strong> {duration_text} • <strong>Processed:</strong> {processed_at[:10]}
            </p>
            
            {f'<div class="episode-link"><a href="{episode_url}" target="_blank">🔗 Listen to Episode</a></div>' if episode_url else ''}
        </div>
        """
    
    html_report += f"""
        </div>
        
        <div class="bottom-line">
            <h2>🎯 Bottom Line</h2>
            <p>AI industry momentum continues with {stats[5] or 0} high-impact developments this week. Key focus areas: strategic AI implementation, market consolidation, and technical breakthroughs driving competitive advantage. <strong>Recommendation:</strong> Monitor developments in {top_companies[0][0] if top_companies else 'leading AI companies'} and prepare for accelerated adoption timelines.</p>
        </div>
        
        <div class="footer">
            <p><strong>AI Intelligence Executive Brief</strong></p>
            <p>🤖 Powered by Claude AI • 📊 JupyterLab Production Environment • 🎯 Executive Communication Optimized</p>
            <p><em>Next briefing: {(datetime.now() + timedelta(days=7)).strftime('%B %d, %Y')}</em></p>
        </div>
    </body>
    </html>
    """
    
    # Generate Enhanced Markdown Report
    markdown_report = f"""# 🎯 AI Intelligence Executive Brief
## Week of {week_range}
*Generated on {report_date}*

---

## 📋 Executive Summary

**Bottom line:** This week's AI intelligence reveals {stats[5] or 0} high-impact developments across {stats[4] or 0} key podcasts.

### 🎯 This Week's Headlines
"""
    
    for headline in headlines:
        if headline:
            markdown_report += f"- **{headline}**\n"
    
    markdown_report += f"""

---

## 📊 Intelligence Metrics

| Metric | Value |
|--------|-------|
| Episodes Analyzed | {stats[0] or 0} |
| High-Impact Stories | {stats[5] or 0} |
| Podcast Sources | {stats[4] or 0} |
| Average Importance | {stats[2] or 0:.1f}/10 |
| Processing Investment | ${stats[1] or 0:.4f} |
| Average Confidence | {stats[3] or 0:.1f} |

---

## 💼 Strategic Implications

"""
    
    for insight in top_strategic:
        if insight:
            markdown_report += f"- **{insight}**\n"
    
    markdown_report += f"""

---

## 🏢 Market Intelligence

### Companies in Focus
"""
    
    for company, count in top_companies[:8]:
        if company:
            markdown_report += f"- **{company}** (mentioned {count} times)\n"
    
    markdown_report += f"""

### Key Industry Voices
"""
    
    for person, count in top_people[:8]:
        if person:
            person_clean = person.split(' - ')[0] if ' - ' in person else person
            markdown_report += f"- **{person_clean}** (mentioned {count} times)\n"
    
    markdown_report += f"""

---

## 📻 Detailed Episode Intelligence

"""
    
    for i, episode in enumerate(episodes[:10], 1):
        (podcast_name, title, pub_date, duration, episode_url, summary, 
         tech_dev, industry, people, companies, predictions, insights, 
         actionable, importance, confidence, cost, processed_at) = episode
        
        # Split headline from summary
        if '|' in summary:
            headline, details = summary.split('|', 1)
            headline = headline.strip()
            details = details.strip()
        else:
            headline = summary[:100] + "..." if len(summary) > 100 else summary
            details = summary
        
        duration_text = f"{duration} min" if duration else "Unknown"
        
        markdown_report += f"""
### {i}. {podcast_name} - {title}

**Impact:** {importance}/10 • **Confidence:** {confidence:.1f} • **Cost:** ${cost:.4f} • **Duration:** {duration_text}

**Headline:** {headline}

**Analysis:** {details}

{f'**Episode Link:** [{episode_url}]({episode_url})' if episode_url else ''}

---

"""
    
    markdown_report += f"""

## 🎯 Bottom Line

AI industry momentum continues with {stats[5] or 0} high-impact developments this week. Key focus areas: strategic AI implementation, market consolidation, and technical breakthroughs driving competitive advantage.

**Recommendation:** Monitor developments in {top_companies[0][0] if top_companies else 'leading AI companies'} and prepare for accelerated adoption timelines.

---

*Next briefing: {(datetime.now() + timedelta(days=7)).strftime('%B %d, %Y')}*

*🤖 Powered by Claude AI & Lily • 📊 JupyterLab Production Environment • 🎯 Executive Communication Optimized*
"""
    
    self.logger.info("Enhanced executive reports generated successfully")
    return html_report, markdown_report


# Add new database schema update method
def update_database_schema(self):
    """Update database schema to include episode URLs"""
    conn = sqlite3.connect(self.db_path)
    cursor = conn.cursor()
    
    try:
        # Check if episode_url column exists
        cursor.execute("PRAGMA table_info(episodes)")
        columns = [column[1] for column in cursor.fetchall()]
        
        if 'episode_url' not in columns:
            cursor.execute('ALTER TABLE episodes ADD COLUMN episode_url TEXT')
            self.logger.info("Added episode_url column to episodes table")
        
        conn.commit()
        conn.close()
        
    except Exception as e:
        self.logger.error(f"Error updating database schema: {e}")
        conn.close()

def calculate_confidence_score(self, transcript: str, intelligence_data: Dict, processing_time: float) -> float:
    """Calculate actual confidence score based on analysis quality"""
    
    confidence_factors = []
    
    # 1. SOURCE QUALITY (0-0.3 points)
    source_quality = self._assess_source_quality(transcript)
    confidence_factors.append(("source_quality", source_quality, 0.3))
    
    # 2. CONTENT COMPLETENESS (0-0.3 points)
    completeness = self._assess_content_completeness(intelligence_data)
    confidence_factors.append(("completeness", completeness, 0.3))
    
    # 3. ANALYSIS DEPTH (0-0.25 points)
    depth = self._assess_analysis_depth(intelligence_data)
    confidence_factors.append(("depth", depth, 0.25))
    
    # 4. PROCESSING INDICATORS (0-0.15 points)
    processing_quality = self._assess_processing_quality(processing_time, intelligence_data)
    confidence_factors.append(("processing", processing_quality, 0.15))
    
    # Calculate weighted score
    total_score = sum(factor_score * weight for _, factor_score, weight in confidence_factors)
    
    # Log confidence breakdown for debugging
    self.logger.debug(f"Confidence breakdown: {confidence_factors} = {total_score:.3f}")
    
    return round(total_score, 3)

def _assess_source_quality(self, transcript: str) -> float:
    """Assess quality of source material (0-1)"""
    if not transcript:
        return 0.0
    
    score = 0.0
    
    # Length check (more content = potentially better analysis)
    if len(transcript) > 2000:
        score += 0.4
    elif len(transcript) > 1000:
        score += 0.3
    elif len(transcript) > 500:
        score += 0.2
    else:
        score += 0.1
    
    # Content indicators (technical terms, business language)
    business_terms = ['strategy', 'market', 'revenue', 'growth', 'competitive', 'innovation', 'industry']
    technical_terms = ['AI', 'machine learning', 'algorithm', 'model', 'data', 'training', 'inference']
    
    business_mentions = sum(1 for term in business_terms if term.lower() in transcript.lower())
    technical_mentions = sum(1 for term in technical_terms if term.lower() in transcript.lower())
    
    # Bonus for relevant terminology
    if business_mentions >= 3:
        score += 0.2
    elif business_mentions >= 1:
        score += 0.1
    
    if technical_mentions >= 3:
        score += 0.2
    elif technical_mentions >= 1:
        score += 0.1
    
    # Content structure indicators
    if '?' in transcript:  # Questions indicate dialogue/interview
        score += 0.1
    if any(indicator in transcript.lower() for indicator in ['discuss', 'explain', 'talk about']):
        score += 0.1
    
    return min(score, 1.0)

def _assess_content_completeness(self, intelligence_data: Dict) -> float:
    """Assess how complete the extracted intelligence is (0-1)"""
    score = 0.0
    
    # Required fields quality
    required_fields = {
        'headline_takeaway': (50, 0.15),  # min length, points
        'executive_summary': (200, 0.2),
        'bottom_line': (30, 0.15)
    }
    
    for field, (min_length, points) in required_fields.items():
        content = intelligence_data.get(field, '')
        if len(content) >= min_length:
            score += points
        elif len(content) >= min_length * 0.5:  # Partial credit
            score += points * 0.5
    
    # Optional but valuable fields
    optional_fields = [
        'strategic_implications', 'technical_developments', 'market_dynamics',
        'key_people', 'companies_mentioned', 'actionable_insights'
    ]
    
    populated_optional = 0
    for field in optional_fields:
        data = intelligence_data.get(field, [])
        if isinstance(data, list) and len(data) > 0:
            # Check if list items have substance
            substantial_items = [item for item in data if len(str(item)) > 20]
            if substantial_items:
                populated_optional += 1
    
    # Bonus for populated optional fields (0-0.3 points)
    optional_score = min(populated_optional / len(optional_fields), 1.0) * 0.3
    score += optional_score
    
    # Quality indicators
    summary = intelligence_data.get('executive_summary', '')
    if len(summary.split('.')) >= 4:  # Multiple sentences
        score += 0.1
    if any(word in summary.lower() for word in ['strategic', 'impact', 'business', 'market']):
        score += 0.1
    
    return min(score, 1.0)

def _assess_analysis_depth(self, intelligence_data: Dict) -> float:
    """Assess depth and quality of analysis (0-1)"""
    score = 0.0
    
    # Check for specific, actionable insights
    insights = intelligence_data.get('actionable_insights', [])
    if isinstance(insights, list):
        # Quality of insights (look for specific language)
        quality_indicators = ['implement', 'evaluate', 'consider', 'monitor', 'assess', 'develop']
        substantial_insights = 0
        
        for insight in insights:
            if len(str(insight)) > 30:  # Substantial length
                if any(indicator in str(insight).lower() for indicator in quality_indicators):
                    substantial_insights += 1
        
        score += min(substantial_insights * 0.15, 0.45)  # Up to 0.45 points
    
    # Check for quantified impacts
    quantified = intelligence_data.get('quantified_impact', [])
    if isinstance(quantified, list) and len(quantified) > 0:
        # Look for actual numbers
        has_numbers = any(any(char.isdigit() for char in str(item)) for item in quantified)
        if has_numbers:
            score += 0.2
        else:
            score += 0.1
    
    # Check for specific company/people mentions with context
    companies = intelligence_data.get('companies_mentioned', [])
    people = intelligence_data.get('key_people', [])
    
    # Quality check - look for context, not just names
    quality_companies = [c for c in companies if ' - ' in str(c) and len(str(c)) > 30]
    quality_people = [p for p in people if ' - ' in str(p) or '(' in str(p)]
    
    if quality_companies or quality_people:
        score += 0.15
    
    # Check importance score reasonableness
    importance = intelligence_data.get('importance_score', 5)
    if 3 <= importance <= 9:  # Reasonable range
        score += 0.1
    
    # Bonus for predictions with timelines or specificity
    predictions = intelligence_data.get('predictions', [])
    if isinstance(predictions, list) and predictions:
        specific_predictions = [p for p in predictions if any(term in str(p).lower() for term in ['2024', '2025', 'months', 'years', 'by'])]
        if specific_predictions:
            score += 0.1
    
    return min(score, 1.0)

def _assess_processing_quality(self, processing_time: float, intelligence_data: Dict) -> float:
    """Assess processing quality indicators (0-1)"""
    score = 0.0
    
    # Processing time indicators
    if 10 <= processing_time <= 60:  # Sweet spot - not too fast (shallow) or slow (struggles)
        score += 0.4
    elif 5 <= processing_time <= 90:
        score += 0.2
    
    # Check for parsing success (no fallback indicators)
    if not intelligence_data.get('parsing_error'):
        score += 0.3
    
    if not intelligence_data.get('fallback_created'):
        score += 0.2
    
    # Cost reasonableness (indicates proper token usage)
    cost = intelligence_data.get('processing_cost', 0)
    if 0.01 <= cost <= 0.10:  # Reasonable cost range
        score += 0.1
    
    return min(score, 1.0)

# ==============================================================================
# STEP 4: Apply the new methods to your system object
# ==============================================================================

# Add these lines at the very end of Cell 3.5, after the existing method assignments:
system.calculate_confidence_score = calculate_confidence_score.__get__(system, ProductionPodcastIntelligence)
system._assess_source_quality = _assess_source_quality.__get__(system, ProductionPodcastIntelligence)
system._assess_content_completeness = _assess_content_completeness.__get__(system, ProductionPodcastIntelligence)
system._assess_analysis_depth = _assess_analysis_depth.__get__(system, ProductionPodcastIntelligence)
system._assess_processing_quality = _assess_processing_quality.__get__(system, ProductionPodcastIntelligence)

print("✅ Confidence scoring system added!")

✅ Confidence scoring system added!


In [11]:
# CELL 4: Process Podcasts
if system:
    print("\n" + "="*70)
    print("🚀 PROCESSING PODCASTS")
    print("="*70)
    
    # Process all configured podcasts
    results = system.process_all_podcasts()
    
    if 'error' in results:
        print(f"❌ Processing failed: {results['error']}")
    else:
        print(f"\n📊 PROCESSING RESULTS:")
        print(f"✅ Episodes processed: {results['processed_episodes']}")
        print(f"❌ Episodes failed: {results['failed_episodes']}")
        print(f"⏭️ Episodes skipped: {results['skipped_episodes']}")
        print(f"💰 Total cost: ${results['total_cost']:.4f}")
        print(f"⏱️ Processing time: {results['batch_processing_time']:.1f} seconds")
        print(f"📻 Podcasts processed: {len(results['podcasts_processed'])}")
        
        if results['episode_details']:
            print(f"\n📋 TOP EPISODES:")
            for episode in results['episode_details'][:3]:
                print(f"📻 {episode['podcast']} - {episode['title'][:50]}...")
                print(f"   ⭐ Importance: {episode['importance']}/10 | 💰 ${episode['cost']:.4f}")


🚀 PROCESSING PODCASTS
2025-09-23 19:43:32,470 - PodcastIntelligence - INFO - === Starting enhanced podcast processing batch ===
2025-09-23 19:43:32,471 - PodcastIntelligence - INFO - Processing 8 active podcasts
2025-09-23 19:43:32,472 - PodcastIntelligence - INFO - Estimated cost: $0.4000
2025-09-23 19:43:32,474 - PodcastIntelligence - INFO - Processing Practical AI (priority: high)
2025-09-23 19:43:32,475 - PodcastIntelligence - INFO - Fetching episodes from Practical AI


/var/folders/wy/mj5hq11s3hd09fwmr3_7mqpw0000gn/T/ipykernel_2621/1079899100.py:566: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''
/var/folders/wy/mj5hq11s3hd09fwmr3_7mqpw0000gn/T/ipykernel_2621/1079899100.py:574: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''


2025-09-23 19:43:35,521 - PodcastIntelligence - INFO - Found 0 new episodes from Practical AI
2025-09-23 19:43:35,523 - PodcastIntelligence - INFO - No new episodes found for Practical AI
2025-09-23 19:43:35,524 - PodcastIntelligence - INFO - Processing Last Week in AI (priority: high)
2025-09-23 19:43:35,525 - PodcastIntelligence - INFO - Fetching episodes from Last Week in AI
2025-09-23 19:43:36,419 - PodcastIntelligence - INFO - Found 1 new episodes from Last Week in AI
2025-09-23 19:43:58,295 - PodcastIntelligence - INFO - Successfully extracted enhanced intelligence from Last Week in AI #322 - Robotaxi progress, OpenAI Business, Gemini in Chrome ($0.0204, 21.9s)
2025-09-23 19:43:58,301 - PodcastIntelligence - INFO - ✅ Processed: Last Week in AI #322 - Robotaxi progress, OpenAI B...
2025-09-23 19:43:58,302 - PodcastIntelligence - INFO - ✅ Last Week in AI: 1 episodes processed in 22.8s
2025-09-23 19:43:58,303 - PodcastIntelligence - INFO - Processing The AI Podcast (priority: medium

In [12]:
# CELL 5: Generate Reports
if system and 'results' in locals() and results.get('processed_episodes', 0) > 0:
    print("\n" + "="*70)
    print("📄 GENERATING INTELLIGENCE REPORTS")
    print("="*70)
    
    # Generate reports
    html_report, markdown_report = system.generate_enhanced_report()
    
    # Save reports
    html_file, md_file = system.save_and_display_reports(html_report, markdown_report)
    
    print(f"✅ Reports generated!")
    print(f"📄 HTML: {html_file}")
    print(f"📝 Markdown: {md_file}")
    
    # Display report
    from IPython.display import HTML, display
    display(HTML(html_report))
    
print("\n🎉 AI Podcast Intelligence System Complete!")


📄 GENERATING INTELLIGENCE REPORTS
2025-09-23 19:45:04,076 - PodcastIntelligence - INFO - Generating enhanced intelligence reports...
2025-09-23 19:45:04,081 - PodcastIntelligence - INFO - Reports saved: AI_Intelligence_Report_20250923_194504.html
✅ Reports generated!
📄 HTML: reports/AI_Intelligence_Report_20250923_194504.html
📝 Markdown: reports/AI_Intelligence_Report_20250923_194504.md



🎉 AI Podcast Intelligence System Complete!


In [13]:
# Cell 4 - Enhanced Processing
if system:
    print("\n" + "="*70)
    print("🚀 PROCESSING WITH ENHANCED CONFIDENCE SCORING")
    print("="*70)
    
    # Process all configured podcasts
    results = system.process_all_podcasts()
    
    if 'error' in results:
        print(f"❌ Processing failed: {results['error']}")
    else:
        print(f"\n📊 ENHANCED PROCESSING RESULTS:")
        print(f"✅ Episodes processed: {results['processed_episodes']}")
        print(f"❌ Episodes failed: {results['failed_episodes']}")
        print(f"⭐ Episodes skipped: {results['skipped_episodes']}")
        print(f"💰 Total cost: ${results['total_cost']:.4f}")
        print(f"⏱️ Processing time: {results['batch_processing_time']:.1f} seconds")
        print(f"📻 Podcasts processed: {len(results['podcasts_processed'])}")
        
        # Show confidence scores
        if results['episode_details']:
            print(f"\n📋 EPISODE INTELLIGENCE WITH CONFIDENCE:")
            for episode in results['episode_details'][:5]:
                confidence = episode.get('confidence', 'N/A')
                print(f"📻 {episode['podcast']} - {episode['title'][:50]}...")
                print(f"   ⭐ Importance: {episode['importance']}/10 | 🎯 Confidence: {confidence:.3f} | 💰 ${episode['cost']:.4f}")
                print(f"   📈 {episode['headline']}")
                print()


🚀 PROCESSING WITH ENHANCED CONFIDENCE SCORING
2025-09-23 19:45:04,093 - PodcastIntelligence - INFO - === Starting enhanced podcast processing batch ===
2025-09-23 19:45:04,094 - PodcastIntelligence - INFO - Processing 8 active podcasts
2025-09-23 19:45:04,095 - PodcastIntelligence - INFO - Estimated cost: $0.4000
2025-09-23 19:45:04,101 - PodcastIntelligence - INFO - Processing Practical AI (priority: high)
2025-09-23 19:45:04,102 - PodcastIntelligence - INFO - Fetching episodes from Practical AI


/var/folders/wy/mj5hq11s3hd09fwmr3_7mqpw0000gn/T/ipykernel_2621/1079899100.py:566: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''
/var/folders/wy/mj5hq11s3hd09fwmr3_7mqpw0000gn/T/ipykernel_2621/1079899100.py:574: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''


2025-09-23 19:45:07,309 - PodcastIntelligence - INFO - Found 0 new episodes from Practical AI
2025-09-23 19:45:07,311 - PodcastIntelligence - INFO - No new episodes found for Practical AI
2025-09-23 19:45:07,311 - PodcastIntelligence - INFO - Processing Last Week in AI (priority: high)
2025-09-23 19:45:07,312 - PodcastIntelligence - INFO - Fetching episodes from Last Week in AI
2025-09-23 19:45:07,631 - PodcastIntelligence - INFO - Found 1 new episodes from Last Week in AI
2025-09-23 19:45:07,635 - PodcastIntelligence - INFO - ⚠️ Last Week in AI: No episodes processed
2025-09-23 19:45:07,635 - PodcastIntelligence - INFO - Processing The AI Podcast (priority: medium)
2025-09-23 19:45:07,636 - PodcastIntelligence - INFO - Fetching episodes from The AI Podcast
2025-09-23 19:45:08,607 - PodcastIntelligence - INFO - Found 1 new episodes from The AI Podcast
2025-09-23 19:45:08,610 - PodcastIntelligence - INFO - ⚠️ The AI Podcast: No episodes processed
2025-09-23 19:45:08,611 - PodcastIntelli

In [14]:
# Cell 5 - Enhanced Reporting
if system and 'results' in locals() and results.get('processed_episodes', 0) > 0:
    print("\n" + "="*70)
    print("📄 GENERATING ENHANCED INTELLIGENCE REPORTS")
    print("="*70)
    
    # Generate reports with confidence scores
    html_report, markdown_report = system.generate_enhanced_report()
    
    # Save reports
    html_file, md_file = system.save_and_display_reports(html_report, markdown_report)
    
    print(f"✅ Enhanced reports generated!")
    print(f"📄 HTML: {html_file}")
    print(f"📝 Markdown: {md_file}")
    
    # Show confidence distribution
    conn = sqlite3.connect(system.db_path)
    confidence_stats = pd.read_sql('''
        SELECT 
            AVG(confidence_score) as avg_confidence,
            MIN(confidence_score) as min_confidence,
            MAX(confidence_score) as max_confidence,
            COUNT(*) as total_episodes
        FROM intelligence 
        WHERE processed_at >= date('now', '-7 days')
    ''', conn)
    conn.close()
    
    print(f"\n🎯 CONFIDENCE SCORE DISTRIBUTION:")
    print(f"📊 Average: {confidence_stats['avg_confidence'].iloc[0]:.3f}")
    print(f"📊 Range: {confidence_stats['min_confidence'].iloc[0]:.3f} - {confidence_stats['max_confidence'].iloc[0]:.3f}")
    print(f"📊 Episodes: {confidence_stats['total_episodes'].iloc[0]}")
    
    # Display the HTML report
    from IPython.display import HTML, display
    display(HTML(html_report))

print("\n🎉 Enhanced AI Podcast Intelligence System Complete!")


🎉 Enhanced AI Podcast Intelligence System Complete!


In [31]:
# Run this to clean up existing duplicates in your database

def cleanup_duplicate_episodes(self):
    """Remove duplicate episodes from database"""
    conn = sqlite3.connect(self.db_path)
    cursor = conn.cursor()
    
    try:
        print("🔍 FINDING DUPLICATE EPISODES...")
        
        # Find potential duplicates
        cursor.execute('''
            SELECT podcast_name, title, COUNT(*) as count, GROUP_CONCAT(id) as ids
            FROM episodes 
            GROUP BY podcast_name, title 
            HAVING COUNT(*) > 1
            ORDER BY podcast_name, title
        ''')
        
        duplicates = cursor.fetchall()
        
        if not duplicates:
            print("✅ No duplicate episodes found!")
            return
        
        print(f"📊 Found {len(duplicates)} sets of duplicate episodes:")
        
        removed_count = 0
        for podcast_name, title, count, ids_str in duplicates:
            episode_ids = [int(id_str) for id_str in ids_str.split(',')]
            print(f"\n📻 {podcast_name}")
            print(f"   📺 {title} ({count} duplicates)")
            
            # Keep the first episode, remove the rest
            keep_id = episode_ids[0]
            remove_ids = episode_ids[1:]
            
            print(f"   ✅ Keeping episode ID: {keep_id}")
            print(f"   🗑️ Removing episode IDs: {remove_ids}")
            
            for remove_id in remove_ids:
                # Remove intelligence data first (foreign key constraint)
                cursor.execute('DELETE FROM intelligence WHERE episode_id = ?', (remove_id,))
                # Remove episode
                cursor.execute('DELETE FROM episodes WHERE id = ?', (remove_id,))
                removed_count += 1
        
        conn.commit()
        print(f"\n✅ Cleanup complete! Removed {removed_count} duplicate episodes.")
        
    except Exception as e:
        print(f"❌ Error during cleanup: {e}")
        conn.rollback()
    finally:
        conn.close()

# Run the cleanup
system.cleanup_duplicate_episodes = cleanup_duplicate_episodes.__get__(system, ProductionPodcastIntelligence)
system.cleanup_duplicate_episodes()

print("🧹 Duplicate cleanup completed!")

🔍 FINDING DUPLICATE EPISODES...
✅ No duplicate episodes found!
🧹 Duplicate cleanup completed!


In [32]:
# IDENTIFY DUPLICATE EPISODES
print("🔍 IDENTIFYING DUPLICATE EPISODES...")

conn = sqlite3.connect(system.db_path)
cursor = conn.cursor()

# Check The AI Podcast episodes specifically
cursor.execute('''
    SELECT e.id, e.title, e.pub_date, e.guid, i.processed_at
    FROM episodes e
    LEFT JOIN intelligence i ON e.id = i.episode_id
    WHERE e.podcast_name = "The AI Podcast"
    ORDER BY e.created_at
''')

ai_podcast_episodes = cursor.fetchall()

print(f"📻 The AI Podcast episodes:")
for ep_id, title, pub_date, guid, processed_at in ai_podcast_episodes:
    print(f"   ID {ep_id}: {title}")
    print(f"      📅 Pub Date: {pub_date}")
    print(f"      🔗 GUID: {guid}")
    print(f"      🧠 Processed: {processed_at}")
    print()

conn.close()

🔍 IDENTIFYING DUPLICATE EPISODES...
📻 The AI Podcast episodes:
   ID 3: From AlphaFold to MMseqs2-GPU: How AI is Accelerating Protein Science
      📅 Pub Date: 2025-09-10T07:00:00
      🔗 GUID: None
      🧠 Processed: 2025-09-12 18:50:53

   ID 8: Bringing Robots to Life with AI: The Three Computer Revolution - Ep. 274
      📅 Pub Date: 2025-09-17T15:54:00
      🔗 GUID: a0dac7d4-932b-11f0-b609-b3a5538e89a1
      🧠 Processed: 2025-09-17 18:25:19

   ID 8: Bringing Robots to Life with AI: The Three Computer Revolution - Ep. 274
      📅 Pub Date: 2025-09-17T15:54:00
      🔗 GUID: a0dac7d4-932b-11f0-b609-b3a5538e89a1
      🧠 Processed: 2025-09-17 18:32:59

